In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, "../src")

In [2]:
import numpy as np
import pandas as pd
import xarray as xr

import fields
import model

In [81]:
params = model.MaternParams(2)

In [84]:
params.get_values()

array([  1. ,   1. ,   1.5,   1.5,   1.5, 500. , 500. , 500. ,   0. ,
         0. ,   0. ])

In [5]:
ds = xr.open_dataset("../data/exp_pro/OCO2_MODIS_4x5deg_monthly_conus_augmented.nc")
ds = ds.where(ds.time.dt.year > 2014, drop=True)

# Format as multi-fields
ds_xco2 = ds[["xco2", "xco2_var"]]
ds_sif = ds[["sif", "sif_var", "evi"]]

datasets = [ds_xco2, ds_sif]
covariates = [["lon", "lat"], ["evi"]]
timedeltas = [0, -1]

In [92]:
mf = fields.MultiField(
        datasets, covariates, np.datetime64("2016-08-01"), timedeltas
    )

In [93]:
df = mf.empirical_variograms(1500, 30)

In [94]:
mod = model.FullBivariateMatern()
mod._composite_wls(df)

TypeError: _composite_wls() missing 1 required positional argument: 'df_vario'

In [95]:
h = np.linspace(10, 1450, 100)
mod.variograms(h)

distance  variogram
i j                           
0 0 0     10.000000   0.000586
    1     24.545455   0.003416
    2     39.090909   0.008381
    3     53.636364   0.015265
    4     68.181818   0.023866
...             ...        ...
1 1 95  1391.818182   0.953106
    96  1406.363636   0.955024
    97  1420.909091   0.956867
    98  1435.454545   0.958638
    99  1450.000000   0.960338

[300 rows x 2 columns]